# Preprocessing SPECFEM3D (Cartesian) for making MTUQ "observed" data
### Félix Rodríguez-Cardozo and Jochen Braunmiller

The following Notebook provides instructions for running the script specfem3D2mtuq.py. This script reads plain text synthetic seismograms created by Specfem3D Cartesian and convert them into SAC files with the format required by MTUQ for input observed data. The aim of such synthetic "observed" data is to run synthetic MTUQ moment tensor estimations. specfem3D2mtuq.py runs in the same environment used for mtuq (conda activate mtuq).  However, you will need to install the **[pyproj](https://anaconda.org/conda-forge/pyproj)** library in the MTUQ environment. 

### 0. Explore the directories


In [ ]:
! ls

 - **2017120102324**: directory with the Specfem3D simulation outcome. Inside that directory, must be placed the DATA and OUTPUT_FILES directories. 
 - **specfem3D2mtuq.py**: script for converting the Specfem3D synthetics into MTUQ observed data. 
 - **MTUQ_TEST**: directory with an example for estimating the seismic moment tensor in MTUQ using the synthetic "observed" data to be pre-processed in this notebook. We will look in detail that directory later. 

In [ ]:
! ls 20171201023244

In [ ]:
#Load the script as a Python library
import specfem3D2mtuq
import importlib
import os
importlib.reload(specfem3D2mtuq)

If you are going to run again this notebook, the next line will remove the PROCESSED directory which is the final result of this manual. If this is your first time here, you can skip the next instruction.

In [ ]:
#Clean space work
specfem3D2mtuq.clean()

### 1. Gathering information about Specfem3D simulation
In this stept the Specfem3D CMTSOLUTION and Par_file files are read to figure out the simulation origin time, hypocenter and whether were used UTM or lat,lon coordinates. In case of using UTM coordinates, the UTM zone used is found as well. You need to provide the path for finding the Specfem3D OUTPUT_FILES directory. For this example, the path is '20171201023244'.

In [ ]:
main_dir = os.getcwd() 
path = '20171201023244'
evla,evlo,evdp,time,ev_id,utm_on,utm_zone = specfem3D2mtuq.get_event_info(path)

### 2. Convert Specfem3D synthetics into MTUQ sac files
In this script each *sem* file in the OUTPUT_FILES directory is read (e.g., IR.MRVT.HXZ.semv) and SAC files in the MTUQ observed data format will be created (e.g., 20171201023244.IR.MRVT..HH.z). You need to provide the path for Specfem3D OUTPUT_FILES directory, the event id and the origin time. The event id is built based on the event origin time: yyyymmddhhmmss, where y is year, m month, d day, h hour, m minute and s second. The origin time must follow the format yyyy-mm-ddThh:mm:ss. The event id and origin time for this example is 20171201023244 and 2017-12-01T02:32:44.0. You can either provide both directly or use the previous step where the method get_event_info returns those values. 

In [ ]:
print('Event ID {}'.format(ev_id))
print('Event origin time {}'.format(time))
specfem3D2mtuq.txt2sac(path,ev_id,time)

**You can see the new created SAC files in MTUQ format, in the directory PROCESSED:**

In [ ]:
!ls PROCESSED/* 

### 3. Gathering information STATIONS file
The new SAC files require some information in their header such as the coordinates of the simulation receivers. Before attempting that step it is necessary to read the STATIONS file used for running the Specfem3D simulation. For collecting the receiver coordinates you need to provide the path, a flag for declaring whether the coordinates in the STATION file are given in UTM format or not, and in case of using UTM coordinates, what is the UTM region. With the aforementioned information, the script converts the coordinates from UTM into latitude and longitude. 

Similarly as step 2, the UTM flag (utm_on) and region (utm_zone) are returned by get_event_info method (stept 1). For this example, the flag is set as True since the simulation was performed in UTM coordinates in the region 38. 

In [ ]:
! cat 20171201023244/DATA/STATIONS

In [ ]:
print(utm_on)
print(utm_zone)

In [ ]:
stations = specfem3D2mtuq.grab_stations(path,utm_on,utm_zone)

**stations** in a list filled with objects of the class **Station** created in specfem3D2mtuq.py. The attributes of each element of the list are: station name (name), station latitude and longitude (lat,lon) and station network (network). In the next cell we will print the attributes for the first element (e.g., station) of the list:

In [ ]:
print(stations[0].name)
print(stations[0].lat)
print(stations[0].lon)
print(stations[0].network)

Observe that the coordinates that were given for the station RST1 in UTM format taking as reference the 38 zone (4130709.407463263 910822.7389732231), now are in lat,long format (37.232399, 49.630001). 

### 4. Completing the SAC header
This step adds the missing header values to the SAC files in the PROCESSED directory. This method is tied with the grab_stations one since one of the input parameters is the stations object created in the previous step. In addition, you need to provide the path for the PROCESSED directory, the event id (ev_id), and the event hypocenter (evla,evlo,evdp). For this example ev_id,evla,evlo, and evdp were determined after using the method get_event_info (stept 1).

In [ ]:
print(ev_id)
print(evla)
print(evlo)
print(evdp)

In [ ]:
process_path = 'PROCESSED'
specfem3D2mtuq.complete_header(process_path,stations,ev_id,evla,evlo,evdp)

### 5. Rotating the radial and transverse

With the SAC headers complete, the next step is to read the seismograms in PROCESSED and rotate them into the radial and transverse components. This method requires the event id (ev_id) and the path to the processed data directory. 

In [ ]:
specfem3D2mtuq.rotate(process_path,ev_id)

After this step, you will see in the PROCESSED directory SAC files corresponding to the radial and transverse seismograms.

In [ ]:
! ls PROCESSED/*.r
! ls PROCESSED/*.t

### 6. Adding zeroes to the trace onset
The synthetics onset is the origin time, However, in some cases, for the receivers closest to the sources, it is convenient to add some zeroes at the beginning of the trace because the proximity of the P-wave arrival. Also, this procedure may be useful when the synthetic tests involve time-shifts. This method requires the path to the processed data, the event id and the time (not samples) to be added to the traces in seconds. For this example, 60s will be addted at the beginning of each trace.

In [ ]:
extra_time = 60
specfem3D2mtuq.padd_zeros(process_path,ev_id,extra_time)

### 7. Scale factor
This method multiply the waveforms amplitude by a constant defined by the user. Our synthetic test expect waveforms in displacement in cm, since Specfem3D output is in m, in our example we multiply the amplitudes by 100. You need to provide the constant (scale) and the path to the PROCESSED directory

In [ ]:
scale = 100
specfem3D2mtuq.scale_amplitude(process_path,scale)

### 8. Writing weights.dat file

Now the SAC files are ready to be used as observed data in MTUQ. For running a synthetic test, the last step is to write the weights.dat input file for running MTUQ. You need to provide the combination of components and the type of waves to include in MTUQ and the path for the PROCESSED data. 

If components = '1 1 1 1 1'. All components and phases will be used. This is: body waves (vertical and radial) and surface waves (vertical, radial, and transverse). 

For this example, the synthetic run will include only surface waves.

In [ ]:
components = '0 0 1 1 1'
specfem3D2mtuq.write_weight_all(process_path,components)

In [ ]:
!cat PROCESSED/weights.dat

### 9. Run MTUQ using Specfem3D Green Functions and the recently created "observed" data 
The last step is to run a synthetic moment tensor estimation using the recently processed observed data. In the directory MTUQ_TEST are already provided the source-side Green Functions for the processed data

**9.1. Copy PROCESSED data into MTUQ_EXAMPLE**

Before going to the MTUQ_TEST directory, you need to copy the PROCESSED directory in the MTUQ_EXAMPLE directory. However, for running the MTUQ example, the directory with the "observed" data has to be named after the event id:

In [ ]:
os.system('cp -r PROCESSED MTUQ_TEST/20171201023244')

**9.2.Exploring the MTUQ_EXAMPLE directory**

In [ ]:
! ls MTUQ_TEST/

- **20171201023244**: Directory with the synthetic "observed" waveforms. 
- **EXPECTED_SOLUTIONS**: Directory with the MTUQ solutions you should obtain after running this example. 
- **GFs**: Source-side Green Functions calculated for the example event using Specfem3D Cartesian. Further information about how to calculate GFs using Specfem3D can be see it in the [cartesian_MT](https://github.com/uafgeotools/mtuq_supp/tree/main/greens_functions_libraries/specfem/cartesian_MT) directory of the [mtuq_supp](https://github.com/uafgeotools/mtuq_supp/tree/main) repository. 
-**SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py,SPECFEM3D_GFs_GridSearch.FMT_SW_options.py**: Scripts for running MTUQ using the Specfem3D synthetic "osberved" data Green Functions (GFs). One script os for estimating the moment tensor restricted to be double-couple and the other is the full moment tensor. In both examples, only surface waves are used. 

**9.3 SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py input parameters required:** 

Now, move the MTUQ_TEST directory and before launching any moment tensor estimation, check the help provided with the SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py script to realize the input parameters required. 

In [ ]:
%cd MTUQ_TEST

In [ ]:
%run -i 'SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py' -h 

The next cell is already set-up for running SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py with the appropiate parameters for the synthetic example. 

In [ ]:
#Launch MTUQ for a double-couple grid-search using the synthetic "observed" data.
%run -i 'SPECFEM3D_GFs_GridSearch.DoubleCouple_SW_options.py' -event 20171201023244 -evla 30.734 -evlo 57.390 -evdp 6000 -mw 4.6 -time 2017-12-01T02:32:44.00000Z -np 51 -fb 15-33

**By running the next cell, you will see the MTUQ grid-search outcome**

In [ ]:
from IPython.display import Image
Image("20171201023244DC_waveforms.png")

## You can compare your results with the one already saved in EXPECTED_SOLUTIONS. END

**P.S. 1:** If you want to start over this notebook, run the next cell for moving to the main directory. 

In [ ]:
os.chdir(main_dir)

**P.S. 1:** If you want to go to the MTUQ_TEST, just for running again the moment tensor estimation, run the next cell. 

In [ ]:
os.chdir('{}/MTUQ_TEST'.format(main_dir))